In [13]:
import numpy as np
import pandas as pd
import json
import matplotlib
import os

from scipy.stats import zscore
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

from numpy import mean,std
from sklearn.model_selection import RepeatedKFold,cross_val_score,KFold,train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,MultiLabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error,accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from keras.utils import normalize, to_categorical
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

import fnmatch
import os

In [14]:
def downsample(df:pd.DataFrame, label_col_name:str) -> pd.DataFrame:
    # find the number of observations in the smallest group
    nmin = df[label_col_name].value_counts().min()
    return (df
            # split the dataframe per group
            .groupby(label_col_name)
            # sample nmin observations from each group
            .apply(lambda x: x.sample(nmin))
            # recombine the dataframes
            .reset_index(drop=True)
            )


In [15]:

path_data = 'ECG_preprocessed_baseline_overlap/'
subjects = os.listdir(path_data)
subjects =[x for x in subjects if x.endswith(".csv")]

df = pd.DataFrame()

for data in subjects:
    
    data = pd.read_csv(path_data+data)
    df = df.append(data, ignore_index=True)
    
# df.to_csv('test.csv')    
#drop measures that are mostly Nan anyway
df.drop(['HRV_SDANN1_diff','HRV_SDNNI1_diff','HRV_SDANN2_diff','HRV_SDNNI2_diff','HRV_SDANN5_diff', \
         'HRV_SDNNI5_diff','HRV_ULF_diff','HRV_VLF_diff','HRV_LF_diff','HRV_VHF_diff', 'HRV_LFHF_diff', \
         'HRV_LFn_diff','HRV_MSE_diff','HRV_CMSE_diff','HRV_RCMSE_diff','HRV_SampEn_diff'],axis=1, inplace=True)  

#dropna based on most basic feature - if this is nan then all other would be 
df.dropna(subset=['HRV_MeanNN_diff', 'HRV_SDNN_diff','HRV_pNN50_diff', 'HRV_pNN20_diff','HR_DIFF'], inplace = True)

In [47]:
'-' in 'PS+'

False

In [48]:
def cond(row):
    if '+' in row['COND']:
        return 1
    elif '-' in row['COND']:
        return 0
    return None
df['COND_BIN'] = df.apply(lambda row: cond(row),axis=1)

In [49]:
df

,TIMESTAMP,ID,COND,IADS-ID,IAPS-ID,ANS-VALENCE,ANS-AROUSAL,ANS-TIME,EVENT,SUB-NAME,ECG,AROUSAL_VALENCE_CAT,AROUSAL_CAT,VALENCE_CAT,HRV_MeanNN_diff,HRV_SDNN_diff,HRV_RMSSD_diff,HRV_SDSD_diff,HRV_CVNN_diff,HRV_CVSD_diff,HRV_MedianNN_diff,HRV_MadNN_diff,HRV_MCVNN_diff,HRV_IQRNN_diff,HRV_pNN50_diff,HRV_pNN20_diff,HRV_HTI_diff,HRV_TINN_diff,HRV_HF_diff,HRV_HFn_diff,HRV_LnHF_diff,HRV_SD1_diff,HRV_SD2_diff,HRV_SD1SD2_diff,HRV_S_diff,HRV_CSI_diff,HRV_CVI_diff,HRV_CSI_Modified_diff,HRV_PIP_diff,HRV_IALS_diff,HRV_PSS_diff,HRV_PAS_diff,HRV_GI_diff,HRV_SI_diff,HRV_AI_diff,HRV_PI_diff,HRV_C1d_diff,HRV_C1a_diff,HRV_SD1d_diff,HRV_SD1a_diff,HRV_C2d_diff,HRV_C2a_diff,HRV_SD2d_diff,HRV_SD2a_diff,HRV_Cd_diff,HRV_Ca_diff,HRV_SDNNd_diff,HRV_SDNNa_diff,HRV_DFA_alpha1_diff,HRV_DFA_alpha1_ExpRange_diff,HRV_DFA_alpha1_ExpMean_diff,HRV_DFA_alpha1_DimRange_diff,HRV_DFA_alpha1_DimMean_diff,HRV_ApEn_diff,HRV_ShanEn_diff,HRV_FuzzyEn_diff,HRV_CD_diff,HRV_HFD_diff,HRV_KFD_diff,HRV_LZC_diff,HR_DIFF,IS_HR_ELEVATED,COND_BIN
0,1.579508e+09,103.0,PS-,711,6315,2.897,6.591,3.133,NaN,SUB103,[24.93610693 24.84740556 24.75629126 ... 6.34...,1,1,0,-8.657143,33.265880,-19.885685,-20.465451,0.045886,-0.025508,-34.5,85.9908,0.120475,126.75,-12.857143,17.619048,-0.500000,-195.3125,NaN,NaN,NaN,-14.471259,61.255350,-0.436389,5299.717123,1.731655,0.130273,1294.093943,-0.190476,-0.197802,-0.500000,0.0,0.828038,0.747438,0.894597,26.373626,0.537332,-0.537332,8.635786,-32.531615,-0.239752,0.239752,23.424523,62.088369,-0.084323,0.084323,17.503967,30.046371,0.500989,-0.522202,0.384370,-0.305138,0.324221,-0.303794,-0.099536,-0.480221,-0.093202,-0.690720,-0.440301,-0.746894,1.019395,1.0,0
1,1.579508e+09,103.0,S-,244,None,2.291,5.625,5.727,NaN,SUB103,[16.04715959 16.13593161 16.22470407 ... 22.67...,1,1,0,-1.933333,-51.587217,-101.439497,-104.939957,-0.070354,-0.138761,-51.0,14.8260,0.027595,-18.00,6.666667,20.000000,0.000000,-445.3125,NaN,NaN,NaN,-74.203756,-44.596771,-0.379972,-42171.793128,1.863337,-0.639053,598.404811,-0.200000,-0.214286,-0.833333,0.0,-8.878358,-8.664644,-9.232320,14.285714,0.125277,-0.125277,-42.725402,-61.446323,-0.182967,0.182967,-49.808981,-11.640301,-0.099897,0.099897,-44.373694,-28.692067,0.678554,-1.193593,0.211233,-0.435634,0.215344,-0.276514,-0.133333,0.115685,0.022273,-0.208479,0.092694,-0.260459,0.960853,0.0,0
3,1.579508e+09,103.0,S-,292,None,2.396,5.569,5.048,NaN,SUB103,[23.45012551 23.45041798 23.44484509 ... 22.81...,1,1,0,-43.533333,22.774254,5.557366,5.724784,0.035932,0.010791,-68.0,1.4826,0.007118,53.00,6.666667,-6.666667,2.500000,-15.6250,NaN,NaN,NaN,4.048034,33.039769,-0.098549,4036.730356,0.752218,0.239669,613.394844,-0.066667,-0.071429,0.300000,0.0,-1.729880,-1.958690,-1.501726,14.285714,0.058304,-0.058304,4.215006,1.332818,-0.195438,0.195438,9.587741,35.339182,-0.173325,0.173325,7.405051,24.135261,0.745486,0.234830,0.820878,0.065061,-0.074593,0.177493,0.133333,-0.258540,0.010563,-0.315730,-0.518458,0.000000,1.066427,1.0,0
4,1.579508e+09,103.0,PS-,709,3170,3.414,6.875,4.808,NaN,SUB103,[26.30257506 26.61242395 26.90708747 ... 6.30...,1,1,0,-46.400000,41.263707,27.207286,28.174671,0.062386,0.041183,-55.5,48.1845,0.074005,78.25,6.190476,10.000000,10.333333,-46.8750,NaN,NaN,NaN,19.922501,56.203932,-0.001149,11829.462110,0.009042,0.532676,634.217148,-0.152381,-0.170330,-0.600000,0.0,3.650180,3.566565,3.716009,17.582418,0.289996,-0.289996,21.876194,3.635287,-0.133920,0.133920,28.981514,49.654100,-0.086235,0.086235,24.921936,34.500137,-0.297981,2.017209,0.209442,0.907758,-0.713812,-0.001682,0.099536,-0.395099,-0.001090,-0.170488,-0.660532,-0.045978,1.076569,1.0,0
5,1.579509e+09,103.0,S+,355,None,6.742,6.077,4.491,NaN,SUB103,[25.22750427 25.16208563 25.08525765 ... 14.72...,3,1,1,-7.687500,-11.648967,-45.412883,-47.022567,-0.015772,-0.064179,-18.5,9.6369,0.015165,14.75,6.250000,18.750000,-2.666667,-148.4375,NaN,NaN,NaN,-33.249976,0.035011,-0.458534,-7574.620774,1.286247,-0.325413,373.584432,-0.062500,-0.066667,-0.466667,0.0,-4.283700,-4.803670,-3.7507

In [50]:
df.dropna(subset=['COND_BIN'], inplace = True)

In [51]:
df

,TIMESTAMP,ID,COND,IADS-ID,IAPS-ID,ANS-VALENCE,ANS-AROUSAL,ANS-TIME,EVENT,SUB-NAME,ECG,AROUSAL_VALENCE_CAT,AROUSAL_CAT,VALENCE_CAT,HRV_MeanNN_diff,HRV_SDNN_diff,HRV_RMSSD_diff,HRV_SDSD_diff,HRV_CVNN_diff,HRV_CVSD_diff,HRV_MedianNN_diff,HRV_MadNN_diff,HRV_MCVNN_diff,HRV_IQRNN_diff,HRV_pNN50_diff,HRV_pNN20_diff,HRV_HTI_diff,HRV_TINN_diff,HRV_HF_diff,HRV_HFn_diff,HRV_LnHF_diff,HRV_SD1_diff,HRV_SD2_diff,HRV_SD1SD2_diff,HRV_S_diff,HRV_CSI_diff,HRV_CVI_diff,HRV_CSI_Modified_diff,HRV_PIP_diff,HRV_IALS_diff,HRV_PSS_diff,HRV_PAS_diff,HRV_GI_diff,HRV_SI_diff,HRV_AI_diff,HRV_PI_diff,HRV_C1d_diff,HRV_C1a_diff,HRV_SD1d_diff,HRV_SD1a_diff,HRV_C2d_diff,HRV_C2a_diff,HRV_SD2d_diff,HRV_SD2a_diff,HRV_Cd_diff,HRV_Ca_diff,HRV_SDNNd_diff,HRV_SDNNa_diff,HRV_DFA_alpha1_diff,HRV_DFA_alpha1_ExpRange_diff,HRV_DFA_alpha1_ExpMean_diff,HRV_DFA_alpha1_DimRange_diff,HRV_DFA_alpha1_DimMean_diff,HRV_ApEn_diff,HRV_ShanEn_diff,HRV_FuzzyEn_diff,HRV_CD_diff,HRV_HFD_diff,HRV_KFD_diff,HRV_LZC_diff,HR_DIFF,IS_HR_ELEVATED,COND_BIN
0,1.579508e+09,103.0,PS-,711,6315,2.897,6.591,3.133,NaN,SUB103,[24.93610693 24.84740556 24.75629126 ... 6.34...,1,1,0,-8.657143,33.265880,-19.885685,-20.465451,0.045886,-0.025508,-34.5,85.9908,0.120475,126.75,-12.857143,17.619048,-0.500000,-195.3125,NaN,NaN,NaN,-14.471259,61.255350,-0.436389,5299.717123,1.731655,0.130273,1294.093943,-0.190476,-0.197802,-0.500000,0.0,0.828038,0.747438,0.894597,26.373626,0.537332,-0.537332,8.635786,-32.531615,-0.239752,0.239752,23.424523,62.088369,-0.084323,0.084323,17.503967,30.046371,0.500989,-0.522202,0.384370,-0.305138,0.324221,-0.303794,-0.099536,-0.480221,-0.093202,-0.690720,-0.440301,-0.746894,1.019395,1.0,0
1,1.579508e+09,103.0,S-,244,None,2.291,5.625,5.727,NaN,SUB103,[16.04715959 16.13593161 16.22470407 ... 22.67...,1,1,0,-1.933333,-51.587217,-101.439497,-104.939957,-0.070354,-0.138761,-51.0,14.8260,0.027595,-18.00,6.666667,20.000000,0.000000,-445.3125,NaN,NaN,NaN,-74.203756,-44.596771,-0.379972,-42171.793128,1.863337,-0.639053,598.404811,-0.200000,-0.214286,-0.833333,0.0,-8.878358,-8.664644,-9.232320,14.285714,0.125277,-0.125277,-42.725402,-61.446323,-0.182967,0.182967,-49.808981,-11.640301,-0.099897,0.099897,-44.373694,-28.692067,0.678554,-1.193593,0.211233,-0.435634,0.215344,-0.276514,-0.133333,0.115685,0.022273,-0.208479,0.092694,-0.260459,0.960853,0.0,0
3,1.579508e+09,103.0,S-,292,None,2.396,5.569,5.048,NaN,SUB103,[23.45012551 23.45041798 23.44484509 ... 22.81...,1,1,0,-43.533333,22.774254,5.557366,5.724784,0.035932,0.010791,-68.0,1.4826,0.007118,53.00,6.666667,-6.666667,2.500000,-15.6250,NaN,NaN,NaN,4.048034,33.039769,-0.098549,4036.730356,0.752218,0.239669,613.394844,-0.066667,-0.071429,0.300000,0.0,-1.729880,-1.958690,-1.501726,14.285714,0.058304,-0.058304,4.215006,1.332818,-0.195438,0.195438,9.587741,35.339182,-0.173325,0.173325,7.405051,24.135261,0.745486,0.234830,0.820878,0.065061,-0.074593,0.177493,0.133333,-0.258540,0.010563,-0.315730,-0.518458,0.000000,1.066427,1.0,0
4,1.579508e+09,103.0,PS-,709,3170,3.414,6.875,4.808,NaN,SUB103,[26.30257506 26.61242395 26.90708747 ... 6.30...,1,1,0,-46.400000,41.263707,27.207286,28.174671,0.062386,0.041183,-55.5,48.1845,0.074005,78.25,6.190476,10.000000,10.333333,-46.8750,NaN,NaN,NaN,19.922501,56.203932,-0.001149,11829.462110,0.009042,0.532676,634.217148,-0.152381,-0.170330,-0.600000,0.0,3.650180,3.566565,3.716009,17.582418,0.289996,-0.289996,21.876194,3.635287,-0.133920,0.133920,28.981514,49.654100,-0.086235,0.086235,24.921936,34.500137,-0.297981,2.017209,0.209442,0.907758,-0.713812,-0.001682,0.099536,-0.395099,-0.001090,-0.170488,-0.660532,-0.045978,1.076569,1.0,0
5,1.579509e+09,103.0,S+,355,None,6.742,6.077,4.491,NaN,SUB103,[25.22750427 25.16208563 25.08525765 ... 14.72...,3,1,1,-7.687500,-11.648967,-45.412883,-47.022567,-0.015772,-0.064179,-18.5,9.6369,0.015165,14.75,6.250000,18.750000,-2.666667,-148.4375,NaN,NaN,NaN,-33.249976,0.035011,-0.458534,-7574.620774,1.286247,-0.325413,373.584432,-0.062500,-0.066667,-0.466667,0.0,-4.283700,-4.803670,-3.7507

In [55]:
#downsample:
df_downsampled = downsample(df, 'COND_BIN')

In [56]:
df_downsampled.shape

(5490, 73)

In [57]:
data = df_downsampled[['HRV_MeanNN_diff', 'HRV_SDNN_diff', 'HRV_RMSSD_diff', 'HRV_SDSD_diff',
       'HRV_CVNN_diff', 'HRV_CVSD_diff', 'HRV_MedianNN_diff', 'HRV_MadNN_diff', 'HRV_MCVNN_diff',
       'HRV_IQRNN_diff', 'HRV_pNN50_diff', 'HRV_pNN20_diff', 'HRV_HTI_diff', 'HRV_TINN_diff',
       'HRV_SD1_diff', 'HRV_SD2_diff', 'HRV_SD1SD2_diff',
       'HRV_CSI_diff', 'HRV_CVI_diff', 'HRV_CSI_Modified_diff', 'HRV_PIP_diff', 'HRV_IALS_diff',
       'HRV_PSS_diff', 'HRV_GI_diff', 'HRV_SI_diff', 'HRV_AI_diff', 'HRV_PI_diff', 'HRV_C1d_diff',
       'HRV_C1a_diff', 'HRV_SD1d_diff', 'HRV_SD1a_diff', 'HRV_C2d_diff', 'HRV_C2a_diff', 'HRV_SD2d_diff',
       'HRV_SD2a_diff', 'HRV_Cd_diff', 'HRV_Ca_diff', 'HRV_SDNNd_diff', 'HRV_SDNNa_diff',
       'HRV_DFA_alpha1_diff', 'HRV_DFA_alpha1_ExpRange_diff', 'HRV_DFA_alpha1_ExpMean_diff',
       'HRV_DFA_alpha1_DimRange_diff', 'HRV_DFA_alpha1_DimMean_diff', 'HRV_ApEn_diff',
       'HRV_ShanEn_diff', 'HRV_FuzzyEn_diff', 'HRV_CD_diff', 'HRV_HFD_diff',
       'HRV_KFD_diff', 'HRV_LZC_diff','HR_DIFF','IS_HR_ELEVATED']]

# data = df_downsampled[['HRV_MeanNN', 'HRV_SDNN','HRV_pNN50', 'HRV_pNN20','HR_DIFF','IS_HR_ELEVATED']]
labels = df_downsampled[['AROUSAL_CAT','VALENCE_CAT','AROUSAL_VALENCE_CAT','COND_BIN']]

In [58]:
pd.set_option('display.max_columns', 100)
data.head(5)

,HRV_MeanNN_diff,HRV_SDNN_diff,HRV_RMSSD_diff,HRV_SDSD_diff,HRV_CVNN_diff,HRV_CVSD_diff,HRV_MedianNN_diff,HRV_MadNN_diff,HRV_MCVNN_diff,HRV_IQRNN_diff,HRV_pNN50_diff,HRV_pNN20_diff,HRV_HTI_diff,HRV_TINN_diff,HRV_SD1_diff,HRV_SD2_diff,HRV_SD1SD2_diff,HRV_CSI_diff,HRV_CVI_diff,HRV_CSI_Modified_diff,HRV_PIP_diff,HRV_IALS_diff,HRV_PSS_diff,HRV_GI_diff,HRV_SI_diff,HRV_AI_diff,HRV_PI_diff,HRV_C1d_diff,HRV_C1a_diff,HRV_SD1d_diff,HRV_SD1a_diff,HRV_C2d_diff,HRV_C2a_diff,HRV_SD2d_diff,HRV_SD2a_diff,HRV_Cd_diff,HRV_Ca_diff,HRV_SDNNd_diff,HRV_SDNNa_diff,HRV_DFA_alpha1_diff,HRV_DFA_alpha1_ExpRange_diff,HRV_DFA_alpha1_ExpMean_diff,HRV_DFA_alpha1_DimRange_diff,HRV_DFA_alpha1_DimMean_diff,HRV_ApEn_diff,HRV_ShanEn_diff,HRV_FuzzyEn_diff,HRV_CD_diff,HRV_HFD_diff,HRV_KFD_diff,HRV_LZC_diff,HR_DIFF,IS_HR_ELEVATED
0,7.467836,-40.583465,-69.865502,-71.879898,-0.067280,-0.115439,19.0,5.9304,0.008759,8.25,-10.526316,-4.970760,2.833333,0.00,-50.826764,-38.260022,-0.603767,1.690255,-1.281847,-4.586270,-0.143275,-0.117647,-0.250000,-4.695194,-5.022746,-4.381649,17.647059,0.082139,-0.082139,-36.290200,-35.624418,-0.224356,0.224356,-31.123485,-23.865330,-0.196968,0.196968,-33.324207,-27.890599,0.179587,-1.732346,-0.479203,-0.493996,0.147909,-0.445239,0.021413,0.444077,0.263525,-0.328040,0.217269,-0.191226,0.982030,0.0
1,-45.777778,-34.273586,-59.913091,-61.742925,-0.049535,-0.088752,-34.0,-12.6021,-0.018233,-13.50,-16.666667,-16.666667,-2.400000,0.00,-43.658841,-28.758601,-0.510600,1.025705,-0.908265,4.037519,-0.055556,-0.091912,-0.206349,0.942177,1.021617,0.865466,3.308824,0.148700,-0.148700,-27.548702,-34.153617,-0.021285,0.021285,-22.043441,-18.490721,0.050060,-0.050060,-24.062557,-25.504258,0.256026,-1.618417,-0.186670,-0.694061,0.597984,-0.268491,-0.597494,0.382144,-0.006781,0.007724,1.064726,0.231663,1.069683,1.0
2,11.258333,-21.254267,-48.538760,-50.100086,-0.031784,-0.070624,17.0,45.9606,0.065622,54.00,1.250000,-4.166667,3.500000,0.00,-35.426110,-13.726144,-0.289806,0.655649,-0.320552,151.545136,-0.045833,-0.042857,-0.066667,4.052585,4.050946,4.024459,18.095238,0.176737,-0.176737,-16.995280,-32.823939,-0.110509,0.110509,-16.740727,-2.613409,-0.019175,0.019175,-16.504169,-14.801915,-0.031243,-2.517574,-1.187217,-0.996820,0.403604,-0.230260,0.156891,0.009316,0.169819,-0.428148,0.127291,-0.208163,0.980834,0.0
3,26.071795,-30.332423,-35.928239,-37.952460,-0.039554,-0.046633,42.0,-13.3434,-0.018718,-10.00,-13.333333,6.153846,-0.666667,-62.50,-26.836442,-38.043193,-0.045422,0.107230,-0.806257,-214.486925,-0.158974,-0.154762,-0.275000,-12.294372,-12.052681,-12.507770,0.000000,-0.051180,0.051180,-15.402095,-21.317613,-0.105793,0.105793,-29.530817,-24.379106,-0.086441,0.086441,-23.550723,-22.864504,0.070159,-1.489726,-0.428440,-0.515151,0.426020,0.062098,-0.360297,0.547550,-0.248908,0.307833,0.221888,-0.115320,0.964807,0.0
4,18.327206,-6.024321,-14.195925,-14.648791,-0.009370,-0.021251,19.0,-0.7413,-0.001582,-3.75,-11.764706,-5.514706,0.600000,-31.25,-10.358259,-3.992748,-0.371586,0.893943,-0.415302,48.117770,-0.088235,-0.042857,-0.111111,-4.378060,-4.312682,-4.442548,3.333333,0.013764,-0.013764,-6.495302,-8.056492,-0.034469,0.034469,-3.505456,-2.041932,0.018909,-0.018909,-4.422064,-4.673697,0.042703,-2.185366,-0.429016,-1.295635,0.668057,-0.088492,-0.102169,0.666637,-0.135821,0.053042,0.303999,0.057366,0.973385,0.0


In [59]:
# HRV_S
# del data['HRV_S']
# del data['HRV_HF']
# data = data.reset_index()
# np.any(np.isnan(np.array(data)))
# col_name = data.columns.to_series()[np.isnan(data).any()]

# del data['HRV_HFn']
# del data['HRV_LnHF']
# np.isinf(data).values.sum()

In [60]:
print(np.isinf(data).values.sum())
print(data.columns.to_series()[np.isnan(data).any()])

0
Series([], dtype: object)


In [61]:
X_train, X_test, y_train, y_test = train_test_split(data, labels['COND_BIN'], test_size=0.2)

In [62]:
# defining parameter range
#Setting C: C is 1 by default and it’s a reasonable default choice. If you have a lot of noisy observations you should decrease it:
# it takes forever for 'linear' kernel+big values of C so not going to try it 
#One is advised to use GridSearchCV with C and gamma spaced exponentially far apart to choose good values.


param_grid = {'C': [0.001,0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001,'scale','auto'],
              'kernel': ['rbf','sigmoid'],
              'class_weight':['balanced',None]} 


grid = GridSearchCV((SVC()), param_grid, cv=5, verbose = 3,n_jobs=-1)
grid.fit(X_train.to_numpy(), y_train.to_numpy())
res = grid.best_estimator_


Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   37.4s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  2.8min finished


In [63]:
y_pred_SVM = grid.predict(X_test.to_numpy())
acc=accuracy_score(y_test.to_numpy(), y_pred_SVM)

print("Results\n\tAccuracy using SVM: {}".format(acc))

Results
	Accuracy using SVM: 0.5


In [64]:
labels['COND'].unique()

KeyError: 'COND'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels['AROUSAL_CAT'], test_size=0.2)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, SGD,RMSprop


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.layers import LSTM, CuDNNLSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
data.shape[1]

In [ ]:
def network(hp):

    
    model = Sequential()
    model.add(Dense(
        hp.Int('input_units', 256, 1024, 256),
        activation='relu',
        input_shape=(data.shape[1],)))
    for i in range(hp.Int('n_layers',1,6)):
        model.add(Dense(
            hp.Int(f'{i}_layer_units',  256, 1024, 256),
            activation='relu',#hp.Choice("activation", ["relu", "tanh"]), 
            ))
    if hp.Boolean("dropout"):
        model.add(Dropout(rate=0.25))
    
    model.add(Dense(1, activation='sigmoid'))
    
    learning_rate = hp.Choice("lr", values=[1e-1, 1e-2, 1e-3])
    adam = Adam(learning_rate=learning_rate)
#     sgd = SGD(learning_rate=learning_rate)
#     rms = RMSprop(learning_rate=learning_rate)  
    
    model.compile(
        loss='binary_crossentropy', 
        optimizer=adam,#keras.optimizers.Adam(learning_rate=learning_rate),
        metrics = ['accuracy'])
    return model


In [ ]:
# from tensorflow import keras
from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
import time


early_stopping = EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True)

tuner = RandomSearch(
    network,
    objective = 'val_accuracy',
    max_trials = 50,
    executions_per_trial = 1,
    directory = os.path.normpath('C:/keras_tuner')
)

tuner.search(x=X_train,
            y=y_train,
            epochs=30,
            batch_size=64,
            callbacks=[early_stopping],
            validation_data=(X_test, y_test)
            )

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hp)

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=50),
         ModelCheckpoint(filepath='best_model_NN.h5', monitor='val_accuracy', save_best_only=True)]

history = model.fit(X_train, y_train, epochs = 100, batch_size = 32,validation_data=(X_test, y_test),callbacks=callbacks)